In [1]:
"""
You need to run this cell for the code in following cells to work.
"""

# Enable module reloading
%load_ext autoreload
%autoreload 2

import os
os.chdir("..")

import pandas as pd
from src.data.pipelines import primary_use_pipeline, square_feet_pipeline, air_temperature_pipeline, \
    dew_temperature_pipeline, sea_level_pressure_pipeline, wind_speed_pipeline, wind_direction_pipeline

In [2]:
def check_mean_and_variance(df_column):
    mean = round(df_column.mean(), 2)
    var = round(df_column.var(), 2)
    print(f'mean = {mean}, variance = {var}')

# Data preprocessing

In [3]:
building_metadata = pd.read_csv('data/building_metadata.csv')
building_metadata.head()

,site_id,building_id,primary_use,square_feet,year_built,floor_count
0,0,0,Education,7432,2008.0,NaN
1,0,1,Education,2720,2004.0,NaN
2,0,2,Education,5376,1991.0,NaN
3,0,3,Education,23685,2002.0,NaN
4,0,4,Education,116607,1975.0,NaN


## Buildings data

### Primary use

In this preprocessing we merge less numerous categories of primary use to category `Other` and encode values using One Hot Encoding.

In [4]:
primary_use_feature = pd.DataFrame(primary_use_pipeline.fit_transform(building_metadata))
primary_use_feature.head()

,0,1,2,3,4,5
0,1.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0


We can see that there is only 6 categories instead of 16 original and values are properly encoded.

### Square feet

In this preprocessing we scale values to zero mean unit variance.

In [5]:
square_feet_feature = pd.DataFrame(square_feet_pipeline.fit_transform(building_metadata))
square_feet_feature.head()

,0
0,-0.764729
1,-0.807282
2,-0.783297
3,-0.617951
4,0.221212


In [6]:
check_mean_and_variance(square_feet_feature[0])

mean = 0.0, variance = 1.0


We can see that are properly scaled.

## Weather data

In [7]:
train_weather = pd.read_csv('data/weather_train.csv')
train_weather

,site_id,timestamp,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,0,2016-01-01 00:00:00,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
1,0,2016-01-01 01:00:00,24.4,NaN,21.1,-1.0,1020.2,70.0,1.5
2,0,2016-01-01 02:00:00,22.8,2.0,21.1,0.0,1020.2,0.0,0.0
3,0,2016-01-01 03:00:00,21.1,2.0,20.6,0.0,1020.1,0.0,0.0
4,0,2016-01-01 04:00:00,20.0,2.0,20.0,-1.0,1020.0,250.0,2.6
...,...,...,...,...,...,...,...,...,...
139768,15,2016-12-31 19:00:00,3.0,NaN,-8.0,NaN,NaN,180.0,5.7
139769,15,2016-12-31 20:00:00,2.8,2.0,-8.9,NaN,1007.4,180.0,7.7
139770,15,2016-12-31 21:00:00,2.8,NaN,-7.2,NaN,1007.5,180.0,5.1
139771,15,2016-12-31 22:00:00,2.2,NaN,-6.7,NaN,1008.0,170.0,4.6


In [8]:
square_feet_feature = pd.DataFrame(square_feet_pipeline.fit_transform(building_metadata))
square_feet_feature.head()

,0
0,-0.764729
1,-0.807282
2,-0.783297
3,-0.617951
4,0.221212


### Air temperature

In this preprocessing we fill in missing values using rolling average and scale values to zero mean unit variance.

In [9]:
air_temperature_feature = pd.DataFrame(air_temperature_pipeline.fit_transform(train_weather))
air_temperature_feature.head()

,0
0,0.995738
1,0.939274
2,0.788704
3,0.628723
4,0.525206


In [10]:
air_temperature_feature[0].isna().sum()

0

In [11]:
check_mean_and_variance(air_temperature_feature[0])

mean = 0.0, variance = 1.0


We can see that there is zero missing values and values are properly scaled.

### Dew temperature

In this preprocessing we fill in missing values using rolling average and scale values to zero mean unit variance.

In [12]:
dew_temperature_feature = pd.DataFrame(dew_temperature_pipeline.fit_transform(train_weather))
dew_temperature_feature.head()

,0
0,1.292475
1,1.404847
2,1.404847
3,1.353769
4,1.292475


In [13]:
dew_temperature_feature[0].isna().sum()

0

In [14]:
check_mean_and_variance(dew_temperature_feature[0])

mean = -0.0, variance = 1.0


We can see that there is zero missing values and values are properly scaled.

### Sea level pressure

In this preprocessing we fill in missing values using rolling average and scale values to zero mean unit variance.

In [15]:
sea_level_pressure_feature = pd.DataFrame(sea_level_pressure_pipeline.fit_transform(train_weather))
sea_level_pressure_feature.head()

,0
0,0.466649
1,0.532188
2,0.532188
3,0.519080
4,0.505972


In [16]:
sea_level_pressure_feature[0].isna().sum()

8755

Since there are still some missing values, we will look into them to find out why they were not filled in with some value.

In [17]:
train_weather[train_weather.sea_level_pressure.isna()].sea_level_pressure.isna().sum()

10618

We can see that although some values were filled in, most were not.

In [18]:
train_weather.groupby(['site_id']).count()

,timestamp,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
site_id,,,,,,,,
0,8784,8781,4954,8781,8783,8699,8534,8784
1,8763,8762,1701,8762,0,8711,8760,8763
2,8783,8782,6429,8782,8717,8739,8192,8778
3,8780,8776,5138,8774,8743,8622,8627,8776
4,8783,8783,4553,8781,7466,8710,8678,8783
5,8755,8753,2725,8753,0,0,8460,8752
6,8782,8772,5790,8772,8782,8607,8003,8743
7,8614,8614,0,8591,731,8606,8614,8614
8,8784,8781,4954,8781,8783,8699,8534,8784


We can see that for 5th site there are no values of sea_level_pressure and therefore we have no way of filling in these data. Also precipitation is missing for that site.

We will probably throw away this attribute or site, because we have no way of filling in data.

In [19]:
check_mean_and_variance(sea_level_pressure_feature[0])

mean = 0.0, variance = 1.0


### Wind speed

In this preprocessing we fill in missing values using rolling average and scale values to zero mean unit variance.

In [20]:
wind_speed_feature = pd.DataFrame(wind_speed_pipeline.fit_transform(train_weather))
wind_speed_feature.head()

,0
0,-1.524278
1,-0.881804
2,-1.524278
3,-1.524278
4,-0.410656


In [21]:
wind_speed_feature[0].isna().sum()

0

In [22]:
check_mean_and_variance(wind_speed_feature[0])

mean = -0.0, variance = 1.0


We can see that there is zero missing values and values are properly scaled.

### Wind direction

In this preprocessing we fill in missing values using rolling average and scale values to zero mean unit variance.

In [23]:
wind_direction_feature = pd.DataFrame(wind_direction_pipeline.fit_transform(train_weather))
wind_direction_feature.head()

,0
0,-1.615903
1,-0.983175
2,-1.615903
3,-1.615903
4,0.643840


In [24]:
wind_direction_feature[0].isna().sum()

0

In [25]:
check_mean_and_variance(wind_direction_feature[0])

mean = 0.0, variance = 1.0


We can see that there is zero missing values and values are properly scaled.